# ADV Setup in Python

In [26]:
import pandas as pd

Read in data files as csv, converting them to dataframes (a pandas table).

Column data type is specified for `OrderDate` because it is not automatically detected.

In [27]:
df_customer = pd.read_csv('DimCustomer.csv')
df_geo = pd.read_csv('DimGeography.csv')
df_product = pd.read_csv('DimProduct.csv')
df_sales = pd.read_csv('FactInternetSales.csv',parse_dates=['OrderDate'])

Define new columns for Target Period (TP)
- `TP_Order_Ind` indicates whether order occured within TP
- `TP_SalesAmt` is the amount of sales in 2008

In [28]:
df_sales['TP_Order_Ind'] = df_sales['OrderDate'] > '2007-12-31'
df_sales['TP_SalesAmt'] = df_sales['TP_Order_Ind'] * df_sales['SalesAmount']
df_sales

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,...,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate,TP_Order_Ind,TP_SalesAmt
0,310,20050701,20050713,20050708,21768,1,19,6,SO43697,1,...,3578.2700,286.2616,89.4568,NaN,NaN,2005-07-01,07/13/2005 12:00:00 AM,07/08/2005 12:00:00 AM,False,0.00
1,346,20050701,20050713,20050708,28389,1,39,7,SO43698,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2005-07-01,07/13/2005 12:00:00 AM,07/08/2005 12:00:00 AM,False,0.00
2,346,20050701,20050713,20050708,25863,1,100,1,SO43699,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2005-07-01,07/13/2005 12:00:00 AM,07/08/2005 12:00:00 AM,False,0.00
3,336,20050701,20050713,20050708,14501,1,100,4,SO43700,1,...,699.0982,55.9279,17.4775,NaN,NaN,2005-07-01,07/13/2005 12:00:00 AM,07/08/2005 12:00:00 AM,False,0.00
4,346,20050701,20050713,20050708,11003,1,6,9,SO43701,1,...,3399.9900,271.9992,84.9998,NaN,NaN,2005-07-01,07/13/2005 12:00:00 AM,07/08/2005 12:00:00 AM,False,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60393,485,20080731,20080812,20080807,15868,1,100,6,SO75122,1,...,21.9800,1.7584,0.5495,NaN,NaN,2008-07-31,08/12/2008 12:00:00 AM,08/07/2008 12:00:00 AM,True,21.98
60394,225,20080731,20080812,20080807,15868,1,100,6,SO75122,2,...,8.9900,0.7192,0.2248,NaN,NaN,2008-07-31,08/12/2008 12:00:00 AM,08/07/2008 12:00:00 AM,True,8.99
60395,485,20080731,20080812,20080807,18759,1,100,6,SO75123,1,...,21.9800,1.7584,0.5495,NaN,NaN,2008-07-31,08/12/2008 12:00:00 AM,08/07/2008 12:00:00 AM,True,21.98
60396,486,20080731,20080812,20080807,18759,1,100,6,SO75123,2,...,159.0000,12.7200,3.9750,NaN,NaN,2008-07-31,08/12/2008 12:00:00 AM,08/07/2008 12:00:00 AM,True,159.00


Aggregate by `CustomerKey` to get total sales per person in 2008.

`Purchase` column indicates whether the customer made a purchase.

In [29]:
df_TP_sales = df_sales.groupby('CustomerKey')['TP_SalesAmt'].agg(TP_Sales='sum', N_Sales='count').reset_index()
df_TP_sales['Purchase'] = df_TP_sales['TP_Sales'] > 0
df_TP_sales.head(20)

,CustomerKey,TP_Sales,N_Sales,Purchase
0,11000,0.00,8,False
1,11001,588.96,11,True
2,11002,0.00,4,False
3,11003,0.00,9,False
4,11004,0.00,6,False
5,11005,0.00,6,False
6,11006,0.00,5,False
7,11007,0.00,8,False
8,11008,0.00,7,False
9,11009,0.00,5,False


Create new columns for Analysis Period (AP), ignoring month of December as Dead Period:
- `AP_Order_Ind` indicates whether order was placed within AP
- `AP07Sales` is the amount of sales in 2007

NOTE: In pandas, ```.copy()``` is added to create a deep copy and not just a reference to the original database.

In [30]:
df_AP_sales = df_sales.loc[df_sales['OrderDate']<'2007-12-01'].copy()
df_AP_sales['AP_Order_Ind'] = df_AP_sales['OrderDate'] >= '2007-1-1'
df_AP_sales['AP07Sales'] = df_AP_sales['SalesAmount'] * df_AP_sales['AP_Order_Ind']
df_AP_sales.sort_values(by='CustomerKey')

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,...,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate,TP_Order_Ind,TP_SalesAmt,AP_Order_Ind,AP07Sales
19223,573,20071104,20071116,20071111,11000,1,6,9,SO57418,1,...,59.6018,NaN,NaN,2007-11-04,11/16/2007 12:00:00 AM,11/11/2007 12:00:00 AM,False,0.0,True,2384.0700
96,344,20050722,20050803,20050729,11000,1,6,9,SO43793,1,...,84.9998,NaN,NaN,2005-07-22,08/03/2005 12:00:00 AM,07/29/2005 12:00:00 AM,False,0.0,False,0.0000
19227,488,20071104,20071116,20071111,11000,1,6,9,SO57418,5,...,1.3498,NaN,NaN,2007-11-04,11/16/2007 12:00:00 AM,11/11/2007 12:00:00 AM,False,0.0,True,53.9900
19226,214,20071104,20071116,20071111,11000,1,6,9,SO57418,4,...,0.8748,NaN,NaN,2007-11-04,11/16/2007 12:00:00 AM,11/11/2007 12:00:00 AM,False,0.0,True,34.9900
19225,530,20071104,20071116,20071111,11000,1,6,9,SO57418,3,...,0.1248,NaN,NaN,2007-11-04,11/16/2007 12:00:00 AM,11/11/2007 12:00:00 AM,False,0.0,True,4.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,346,20060121,20060202,20060128,29476,1,100,8,SO45199,1,...,84.9998,NaN,NaN,2006-01-21,02/02/2006 12:00:00 AM,01/28/2006 12:00:00 AM,False,0.0,False,0.0000
4274,358,20070308,20070320,20070315,29479,1,100,7,SO49617,1,...,51.2275,NaN,NaN,2007-03-08,03/20/2007 12:00:00 AM,03/15/2007 12:00:00 AM,False,0.0,True,2049.0982
1283,349,20060213,20060225,20060220,29481,1,100,8,SO45427,1,...,84.3748,NaN,NaN,2006-02-13,02/25/2006 12:00:00 AM,02/20/2006 12:00:00 AM,False,0.0,False,0.0000
4403,358,20070322,20070403,20070329,29482,1,100,7,SO49746,1,...,51.2275,NaN,NaN,2007-03-22,04/03/2007 12:00:00 AM,03/29/2007 12:00:00 AM,False,0.0,True,2049.0982


Group AP by `CustomerKey` to get customer-level data.

`nunique` counts the number of unique values in `SalesOrderNumber` and allows us to aggregate directly from item to customer-level (skipping order-level).

In [31]:
df_AP_sales = df_AP_sales[['CustomerKey','OrderDate','SalesOrderNumber','SalesAmount','AP07Sales']].groupby('CustomerKey').agg(
    LastOrder=pd.NamedAgg(column='OrderDate', aggfunc='max'),
    FirstOrder=pd.NamedAgg(column='OrderDate', aggfunc='min'),
    NumOrders=pd.NamedAgg(column='SalesOrderNumber', aggfunc='nunique'),
    M_Tot_Sales=pd.NamedAgg(column="SalesAmount", aggfunc='sum'),
    M_AP07_Sales=pd.NamedAgg(column='AP07Sales', aggfunc='sum')).reset_index()
df_AP_sales

,CustomerKey,LastOrder,FirstOrder,NumOrders,M_Tot_Sales,M_AP07_Sales
0,11000,2007-11-04,2005-07-22,3,8248.9900,4849.0000
1,11001,2007-07-20,2005-07-18,2,5794.9200,2419.9300
2,11002,2007-08-27,2005-07-10,3,8114.0400,4714.0500
3,11003,2007-11-11,2005-07-01,3,8139.2900,4739.3000
4,11004,2007-11-02,2005-07-26,3,8196.0100,4796.0200
...,...,...,...,...,...,...
10173,29476,2006-01-21,2006-01-21,1,3399.9900,0.0000
10174,29479,2007-03-08,2007-03-08,1,2049.0982,2049.0982
10175,29481,2006-02-13,2006-02-13,1,3374.9900,0.0000
10176,29482,2007-03-22,2007-03-22,1,2049.0982,2049.0982


Add Recency and Frequency columns to AP table
- `R_Last` is days since last purchase
- `First` is days since first purchase
- `F_AvgDays` is the average days between purchase

In [32]:
df_AP_sales['R_Last'] = (pd.to_datetime('2007-11-30') - df_AP_sales['LastOrder']).dt.days
df_AP_sales['First'] = (pd.to_datetime('2007-11-30') - df_AP_sales['FirstOrder']).dt.days
df_AP_sales['F_AvgDays'] = (df_AP_sales['First'] - df_AP_sales['R_Last']) / df_AP_sales['NumOrders']
df_AP_sales

,CustomerKey,LastOrder,FirstOrder,NumOrders,M_Tot_Sales,M_AP07_Sales,R_Last,First,F_AvgDays
0,11000,2007-11-04,2005-07-22,3,8248.9900,4849.0000,26,861,278.333333
1,11001,2007-07-20,2005-07-18,2,5794.9200,2419.9300,133,865,366.000000
2,11002,2007-08-27,2005-07-10,3,8114.0400,4714.0500,95,873,259.333333
3,11003,2007-11-11,2005-07-01,3,8139.2900,4739.3000,19,882,287.666667
4,11004,2007-11-02,2005-07-26,3,8196.0100,4796.0200,28,857,276.333333
...,...,...,...,...,...,...,...,...,...
10173,29476,2006-01-21,2006-01-21,1,3399.9900,0.0000,678,678,0.000000
10174,29479,2007-03-08,2007-03-08,1,2049.0982,2049.0982,267,267,0.000000
10175,29481,2006-02-13,2006-02-13,1,3374.9900,0.0000,655,655,0.000000
10176,29482,2007-03-22,2007-03-22,1,2049.0982,2049.0982,253,253,0.000000


Merge TP columns into AP, keeping only the customers in common with AP, and drop extra columns.

In [33]:
df_ADV = df_AP_sales.merge(df_TP_sales,how='left',on='CustomerKey').drop(columns=['N_Sales','LastOrder','FirstOrder'])
df_ADV

,CustomerKey,NumOrders,M_Tot_Sales,M_AP07_Sales,R_Last,First,F_AvgDays,TP_Sales,Purchase
0,11000,3,8248.9900,4849.0000,26,861,278.333333,0.00,False
1,11001,2,5794.9200,2419.9300,133,865,366.000000,588.96,True
2,11002,3,8114.0400,4714.0500,95,873,259.333333,0.00,False
3,11003,3,8139.2900,4739.3000,19,882,287.666667,0.00,False
4,11004,3,8196.0100,4796.0200,28,857,276.333333,0.00,False
...,...,...,...,...,...,...,...,...,...
10173,29476,1,3399.9900,0.0000,678,678,0.000000,0.00,False
10174,29479,1,2049.0982,2049.0982,267,267,0.000000,0.00,False
10175,29481,1,3374.9900,0.0000,655,655,0.000000,0.00,False
10176,29482,1,2049.0982,2049.0982,253,253,0.000000,0.00,False
